# Assignment 2: Classification 

For this assignment, you have three tasks on the dataset penguins (with missing data). 
- Task 1: Feature engineering (20%)
- Task 2: Implement and try different classification techniques (40%)
- Task 3: Hyperparameter tuning via cross validation, using Randomized Search when applicable (40%)


In [1]:
# Load necessary libraries
# Do not change the code in this section!
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.utils.fixes import loguniform
from scipy.stats import uniform
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import pandas as pd
import seaborn
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')


df  = seaborn.load_dataset("penguins")
print("Penguin dataset")
print(df[:5])

X=df[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g"]].to_numpy() # four numeric features
y=pd.Categorical(df["species"], categories=df["species"].unique()).codes # label are penguin species
print("Features we use for this assignment")
print(X[:5,:])
print(y[:5])


Penguin dataset
  species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
3  Adelie  Torgersen             NaN            NaN                NaN   
4  Adelie  Torgersen            36.7           19.3              193.0   

   body_mass_g     sex  
0       3750.0    Male  
1       3800.0  Female  
2       3250.0  Female  
3          NaN     NaN  
4       3450.0  Female  
Features we use for this assignment
[[  39.1   18.7  181.  3750. ]
 [  39.5   17.4  186.  3800. ]
 [  40.3   18.   195.  3250. ]
 [   nan    nan    nan    nan]
 [  36.7   19.3  193.  3450. ]]
[0 0 0 0 0]


## Task 1: Feature engineering (20%)
- Current X may trigger errors due to the missing data. Try to fix this issue. (10%)
    - Hint: check sklearn.impute.SimpleImputer, for example.
- You can design your own feature engineering pipeline. You are free to use anything you think could help. (10%)


In [2]:
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [3]:
pd.crosstab(index = df['species'], columns = df['island'])

island,Biscoe,Dream,Torgersen
species,,,
Adelie,44,56,52
Chinstrap,0,68,0
Gentoo,124,0,0


In [4]:
# Considering the different species of penguins, different habitat islands, and different genders, 
# when filling missing values, it should be filled in layers
sex=df['sex'].values.reshape(-1,1)  
imp_mean=SimpleImputer(missing_values=np.nan,strategy='most_frequent') 
imp_mean=imp_mean.fit_transform(sex)  
df['sex']=imp_mean
df['bill_length_mm'] = df.groupby(['species','island','sex'])['bill_length_mm'].transform(lambda v: v.fillna(v.mean()))
df['bill_depth_mm'] = df.groupby(['species','island','sex'])['bill_depth_mm'].transform(lambda v: v.fillna(v.mean()))
df['flipper_length_mm'] = df.groupby(['species','island','sex'])['flipper_length_mm'].transform(lambda v: v.fillna(v.mean()))
df['body_mass_g'] = df.groupby(['species','island','sex'])['body_mass_g'].transform(lambda v: v.fillna(v.mean()))

In [5]:
df.isnull().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

In [6]:
X=df[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g"]].to_numpy()
feature=make_pipeline(StandardScaler())
feature.fit(X)
X=feature.transform(X)

## Task 2: Implement and try different classification techniques (40%)

- Implement the following classification methods (with DEFAULT parameters, do not set any parameters for the classifiers in this task) and print 5-folds cross validation results on Macro F1 score. Note that in all the tasks in this assignment, we will use 5-fold cross validation score on Macro F1 as our metric. We do not split training and test datasets. So you are required to use all data for training. (30%)
    - Logistic Regression: LogisticRegression()
    - Decision Tree: DecisionTreeClassifier()
    - Gradient Boosting: GradientBoostingClassifier()
    - Random Forest: RandomForestClassifier()
    - Naive Bayes: GaussianNB()
    - Support Vector Machine: SVC()
    - K Nearest Neighbor: KNeighborsClassifier()
    
- Compute and print the averaged Macro F1 score for all above classifiers (with untuned parameters). Better score will receive more points! (10%)
    

In [7]:
X_train, y_train = X, y
# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
cv_scores_lr = cross_val_score(lr, X_train, y_train, cv=5, scoring='f1_macro')
LogisticRegression_CV_score = np.mean(cv_scores_lr)
print("CV score of Logistic Regression is %f" % LogisticRegression_CV_score)

# Decision Tree
dt = DecisionTreeClassifier() 
dt.fit(X_train, y_train)
cv_scores_dt = cross_val_score(dt, X_train, y_train, cv=5, scoring='f1_macro')
DecisionTree_CV_score = np.mean(cv_scores_dt)
print("CV score of Decision Tree is %f" % DecisionTree_CV_score)

# Gradient Boosting
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
cv_scores_gb = cross_val_score(gb, X_train, y_train, cv=5, scoring='f1_macro')
GradientBoosting_CV_score = np.mean(cv_scores_gb)
print("CV score of Gradient Boosting is %f" % GradientBoosting_CV_score)

# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
cv_scores_rf = cross_val_score(rf, X_train, y_train, cv=5, scoring='f1_macro')
RandomForest_CV_score = np.mean(cv_scores_rf)
print("CV score of Random Forest is %f" % RandomForest_CV_score)

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
cv_scores_nb = cross_val_score(nb, X_train, y_train, cv=5, scoring='f1_macro')
NaiveBayes_CV_score = np.mean(cv_scores_nb)
print("CV score of Naive Bayes is %f" % NaiveBayes_CV_score)

# Support Vector Machine
svm = SVC()
svm.fit(X_train, y_train)
cv_scores_svm = cross_val_score(svm, X_train, y_train, cv=5, scoring='f1_macro')
SupportVectorMachine_CV_score = np.mean(cv_scores_svm)
print("CV score of Support Vector Machine is %f" % SupportVectorMachine_CV_score)

# K Nearest Neighbor
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
cv_scores_knn = cross_val_score(knn, X_train, y_train, cv=5, scoring='f1_macro')
KNearestNeighbor_CV_score = np.mean(cv_scores_knn)
print("CV score of K Nearest Neighbor is %f" % KNearestNeighbor_CV_score)

CV score of Logistic Regression is 0.989291
CV score of Decision Tree is 0.972844
CV score of Gradient Boosting is 0.975820
CV score of Random Forest is 0.972579
CV score of Naive Bayes is 0.964164
CV score of Support Vector Machine is 0.978605
CV score of K Nearest Neighbor is 0.988988


In [8]:
avg =[LogisticRegression_CV_score,DecisionTree_CV_score,GradientBoosting_CV_score,RandomForest_CV_score,NaiveBayes_CV_score,SupportVectorMachine_CV_score,KNearestNeighbor_CV_score]
cv_scores_average_all_classiiers = np.mean(avg)
print("Average CV scores for all seven classifiers is %f" % (cv_scores_average_all_classiiers))

Average CV scores for all seven classifiers is 0.977470


## Task 3: Hyperparameter tuning via cross validation, using Randomized Search when applicable (40%)

- Hyperparameter tuning (Part1~7) using the 5-fold cross validation score on Macro F1 (set scoring='f1_macro' in the code) with the number of iterations for randomized searching n_iter=100. (30%)
    - hint: Sometimes you do not need Randomized Search. Because Grid Search is enough and even faster.

- You are free to fine-tune any hyperparameters (to any one of the seven classifiers) in Part 8, and try to have better performance on 5-fold cross validation score on Macro F1. Better score will receive more points! (10%)


### Part 1: Tuning Logistic Regression 
- Solvers: 'lbfgs', 'liblinear'
- C values in log scale from 1e-2 (0.01) to 1e2 (100)

In [9]:
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV

lr = LogisticRegression()
dist = {'C': loguniform(1e-2, 1e2), 'solver': ['lbfgs', 'liblinear']}
rand_search = RandomizedSearchCV(lr, param_distributions=dist, cv=5, scoring='accuracy',n_iter=100)
rand_result = rand_search.fit(X_train, y_train)

print("Best: %f using %s" % (rand_search.best_score_, rand_search.best_params_))
means = rand_result.cv_results_['mean_test_score']
stds = rand_result.cv_results_['std_test_score']
params = rand_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.991304 using {'C': 5.167308608272073, 'solver': 'liblinear'}
0.985422 (0.009301) with: {'C': 1.6236109888366081, 'solver': 'liblinear'}
0.991304 (0.007100) with: {'C': 5.167308608272073, 'solver': 'liblinear'}
0.944714 (0.011190) with: {'C': 0.017635138478045747, 'solver': 'liblinear'}
0.970887 (0.015994) with: {'C': 0.04154896372650194, 'solver': 'liblinear'}
0.982523 (0.016961) with: {'C': 0.4248441963926956, 'solver': 'lbfgs'}
0.970887 (0.015994) with: {'C': 0.05951591786684235, 'solver': 'liblinear'}
0.976726 (0.011638) with: {'C': 0.08366683562632657, 'solver': 'liblinear'}
0.991304 (0.011594) with: {'C': 7.3861966649189155, 'solver': 'lbfgs'}
0.982523 (0.010937) with: {'C': 1.2256530726441102, 'solver': 'liblinear'}
0.988406 (0.010845) with: {'C': 7.966931296129519, 'solver': 'lbfgs'}
0.979625 (0.011659) with: {'C': 0.09235026088877112, 'solver': 'lbfgs'}
0.988406 (0.010845) with: {'C': 46.75415925901142, 'solver': 'liblinear'}
0.982523 (0.010937) with: {'C': 0.2157291336

In [10]:
lr = LogisticRegression(solver='liblinear', C=5.167308608272073)
lr.fit(X_train, y_train)
cv_scores_lr = cross_val_score(lr, X_train, y_train, cv=5, scoring='f1_macro')
LogisticRegression_CV_score = np.mean(cv_scores_lr)
print("CV score of Logistic Regression is %f" % LogisticRegression_CV_score)

CV score of Logistic Regression is 0.988988


### Part 2:  Tuning Decision Tree with pruning
- ccp_alphas, hint: how to get the candidate set of ccp_alphas?

In [11]:
dt = DecisionTreeClassifier()
path = dt.cost_complexity_pruning_path(X_train, y_train)
grid = dict(ccp_alpha = path.ccp_alphas)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
grid_search = GridSearchCV(dt, param_grid=grid, cv=cv, scoring='accuracy')
grid_result = grid_search.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.960336 using {'ccp_alpha': 0.0023255813953488367}
0.959300 (0.030651) with: {'ccp_alpha': 0.0}
0.960336 (0.032177) with: {'ccp_alpha': 0.0023255813953488367}
0.960196 (0.032332) with: {'ccp_alpha': 0.002886212624584718}
0.954454 (0.029667) with: {'ccp_alpha': 0.004069767441860466}
0.954454 (0.034970) with: {'ccp_alpha': 0.008305647840531562}
0.956443 (0.037266) with: {'ccp_alpha': 0.009205426356589146}
0.950532 (0.036917) with: {'ccp_alpha': 0.011167383555917353}
0.939076 (0.040454) with: {'ccp_alpha': 0.030647840531561457}
0.848880 (0.064703) with: {'ccp_alpha': 0.20728547589951898}
0.622157 (0.171706) with: {'ccp_alpha': 0.3342711139641028}


In [12]:
dt = DecisionTreeClassifier(ccp_alpha = 0.0023255813953488367) 
dt.fit(X_train, y_train)
cv_scores_dt = cross_val_score(dt, X_train, y_train, cv=5, scoring='f1_macro')
DecisionTree_CV_score = np.mean(cv_scores_dt)
print("CV score of Decision Tree is %f" % DecisionTree_CV_score)

CV score of Decision Tree is 0.947906


### Part 3: Tuning Gradient Boosting 
- n_estimators: from range 1 to 100, hint: using 'n_estimators': range(1, 100)
- max_depth': from range 1 to 5 

In [13]:
gb = GradientBoostingClassifier()

dist = {'n_estimators': range(1, 100), 'max_depth': range(1, 5)}
rand_search = RandomizedSearchCV(gb, param_distributions=dist, cv=5, scoring='accuracy',n_iter=100)
rand_result = rand_search.fit(X_train, y_train)

print("Best: %f using %s" % (rand_search.best_score_, rand_search.best_params_))
means = rand_result.cv_results_['mean_test_score']
stds = rand_result.cv_results_['std_test_score']
params = rand_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.988363 using {'n_estimators': 81, 'max_depth': 4}
0.979710 (0.014780) with: {'n_estimators': 85, 'max_depth': 1}
0.979625 (0.017435) with: {'n_estimators': 80, 'max_depth': 3}
0.982566 (0.016894) with: {'n_estimators': 82, 'max_depth': 3}
0.979710 (0.014780) with: {'n_estimators': 69, 'max_depth': 1}
0.973870 (0.019201) with: {'n_estimators': 36, 'max_depth': 2}
0.976769 (0.017370) with: {'n_estimators': 67, 'max_depth': 2}
0.970929 (0.009168) with: {'n_estimators': 31, 'max_depth': 4}
0.979668 (0.014763) with: {'n_estimators': 42, 'max_depth': 2}
0.950597 (0.026854) with: {'n_estimators': 13, 'max_depth': 2}
0.979668 (0.014763) with: {'n_estimators': 98, 'max_depth': 3}
0.956394 (0.012965) with: {'n_estimators': 25, 'max_depth': 1}
0.976726 (0.014814) with: {'n_estimators': 59, 'max_depth': 3}
0.985465 (0.012963) with: {'n_estimators': 66, 'max_depth': 4}
0.973870 (0.014165) with: {'n_estimators': 59, 'max_depth': 2}
0.970972 (0.018299) with: {'n_estimators': 85, 'max_depth': 

In [14]:
gb = GradientBoostingClassifier(n_estimators=81, max_depth=4)
gb.fit(X_train, y_train)
cv_scores_gb = cross_val_score(gb, X_train, y_train, cv=5, scoring='f1_macro')
GradientBoosting_CV_score = np.mean(cv_scores_gb)
print("CV score of Gradient Boosting is %f" % GradientBoosting_CV_score)

CV score of Gradient Boosting is 0.983333


### Part 4: Tuning Random Forest (hint: if too slow, use parallel computation, n_jobs=-1)
- n_estimators: from range 1 to 100
- max_depth': from range 1 to 5 



In [15]:
rf = RandomForestClassifier(n_jobs=-1)

dist = {'n_estimators': range(1, 100), 'max_depth': range(1, 5)}
rand_search = RandomizedSearchCV(rf, param_distributions=dist, cv=5, scoring='accuracy',n_iter=100)
rand_result = rand_search.fit(X_train, y_train)

print("Best: %f using %s" % (rand_search.best_score_, rand_search.best_params_))
means = rand_result.cv_results_['mean_test_score']
stds = rand_result.cv_results_['std_test_score']
params = rand_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.979668 using {'n_estimators': 19, 'max_depth': 4}
0.962234 (0.014731) with: {'n_estimators': 85, 'max_depth': 3}
0.962234 (0.014731) with: {'n_estimators': 71, 'max_depth': 3}
0.799361 (0.011895) with: {'n_estimators': 87, 'max_depth': 1}
0.796462 (0.013719) with: {'n_estimators': 19, 'max_depth': 1}
0.799361 (0.011895) with: {'n_estimators': 79, 'max_depth': 1}
0.970887 (0.013107) with: {'n_estimators': 66, 'max_depth': 4}
0.959292 (0.014228) with: {'n_estimators': 56, 'max_depth': 3}
0.970929 (0.009168) with: {'n_estimators': 5, 'max_depth': 4}
0.953538 (0.021231) with: {'n_estimators': 41, 'max_depth': 2}
0.965132 (0.011553) with: {'n_estimators': 75, 'max_depth': 3}
0.962234 (0.014731) with: {'n_estimators': 72, 'max_depth': 2}
0.956394 (0.015878) with: {'n_estimators': 80, 'max_depth': 2}
0.967988 (0.014306) with: {'n_estimators': 90, 'max_depth': 4}
0.947698 (0.028362) with: {'n_estimators': 13, 'max_depth': 2}
0.973785 (0.011052) with: {'n_estimators': 84, 'max_depth': 4

In [16]:
rf = RandomForestClassifier(n_estimators=19, max_depth=4)
rf.fit(X_train, y_train)
cv_scores_rf = cross_val_score(rf, X_train, y_train, cv=5, scoring='f1_macro')
RandomForest_CV_score = np.mean(cv_scores_rf)
print("CV score of Random Forest is %f" % RandomForest_CV_score)

CV score of Random Forest is 0.961771


### Part 5: Nothing to tune for Naive Bayes
- Just compute cross validate score 

In [17]:
nb = GaussianNB()
nb.fit(X_train, y_train)
cv_scores_nb = cross_val_score(nb, X_train, y_train, cv=5, scoring='f1_macro')
NaiveBayes_CV_score = np.mean(cv_scores_nb)
print("CV score of Naive Bayes is %f" % NaiveBayes_CV_score)

CV score of Naive Bayes is 0.964164


### Part 6: Tuning Support Vector Machine
- kernel: 'linear', 'poly', 'rbf'
- C: from logarithmic range 1e-2 to 1e2



In [18]:
svm = SVC()
dist = {'C': loguniform(1e-2, 1e2), 'kernel': ['linear', 'poly', 'rbf']}
rand_search = RandomizedSearchCV(svm, param_distributions=dist, cv=5, scoring='accuracy',n_iter=100)
rand_result = rand_search.fit(X_train, y_train)

print("Best: %f using %s" % (rand_search.best_score_, rand_search.best_params_))
means = rand_result.cv_results_['mean_test_score']
stds = rand_result.cv_results_['std_test_score']
params = rand_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.991304 using {'C': 15.546645556090532, 'kernel': 'rbf'}
0.979668 (0.014763) with: {'C': 18.82485346441966, 'kernel': 'poly'}
0.938960 (0.019204) with: {'C': 0.47434119416178094, 'kernel': 'poly'}
0.985422 (0.013059) with: {'C': 0.2960049489921704, 'kernel': 'linear'}
0.985465 (0.009166) with: {'C': 4.569483147082367, 'kernel': 'rbf'}
0.982523 (0.010937) with: {'C': 38.626333898839455, 'kernel': 'linear'}
0.976726 (0.014814) with: {'C': 0.1281404941852073, 'kernel': 'rbf'}
0.979625 (0.014831) with: {'C': 0.07469217142681027, 'kernel': 'linear'}
0.979625 (0.011659) with: {'C': 64.27942916855599, 'kernel': 'linear'}
0.982523 (0.010937) with: {'C': 14.425288993595567, 'kernel': 'linear'}
0.982523 (0.010937) with: {'C': 43.12922840174292, 'kernel': 'linear'}
0.962191 (0.007208) with: {'C': 1.2314291782963311, 'kernel': 'poly'}
0.982523 (0.010937) with: {'C': 1.659993508175006, 'kernel': 'rbf'}
0.970929 (0.015877) with: {'C': 0.023991401364988413, 'kernel': 'linear'}
0.985465 (0.0091

In [19]:
svm = SVC(kernel = 'rbf', C = 15.546645556090532)
svm.fit(X_train, y_train)
cv_scores_svm = cross_val_score(svm, X_train, y_train, cv=5, scoring='f1_macro')
SupportVectorMachine_CV_score = np.mean(cv_scores_svm)
print("CV score of Support Vector Machine is %f" % SupportVectorMachine_CV_score)

CV score of Support Vector Machine is 0.989121


### Part 7:  Tuning K Nearest Neighbor
- n_neighbors: from range 1 to 10


In [20]:
knn = KNeighborsClassifier()
dict = {'n_neighbors': range(1,10)}
rand_search = RandomizedSearchCV(knn, param_distributions=dict, cv=5, scoring='accuracy',n_iter=100)
rand_result = rand_search.fit(X_train, y_train)

print("Best: %f using %s" % (rand_search.best_score_, rand_search.best_params_))
means = rand_result.cv_results_['mean_test_score']
stds = rand_result.cv_results_['std_test_score']
params = rand_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.991304 using {'n_neighbors': 5}
0.982609 (0.023188) with: {'n_neighbors': 1}
0.979668 (0.014763) with: {'n_neighbors': 2}
0.988406 (0.010845) with: {'n_neighbors': 3}
0.988363 (0.005819) with: {'n_neighbors': 4}
0.991304 (0.007100) with: {'n_neighbors': 5}
0.988363 (0.005819) with: {'n_neighbors': 6}
0.988363 (0.005819) with: {'n_neighbors': 7}
0.985465 (0.009166) with: {'n_neighbors': 8}
0.985465 (0.009166) with: {'n_neighbors': 9}


In [21]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)
cv_scores_knn = cross_val_score(knn, X_train, y_train, cv=5, scoring='f1_macro')
KNearestNeighbor_CV_score = np.mean(cv_scores_knn)
print("CV score of K Nearest Neighbor is %f" % KNearestNeighbor_CV_score)

CV score of K Nearest Neighbor is 0.988988


### Part 8: Even higher performance!



In [22]:
# You can try many settings offline, but only put one final model for your final submission. 
# TA will evaluate ONLY ONE model here.
svm = SVC(kernel = 'rbf', C = 15.546645556090532)
svm.fit(X_train, y_train)
cv_scores_svm = cross_val_score(svm, X_train, y_train, cv=5, scoring='f1_macro')
SupportVectorMachine_CV_score = np.mean(cv_scores_svm)
print("CV score of Support Vector Machine is %f" % SupportVectorMachine_CV_score)

CV score of Support Vector Machine is 0.989121
